In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,755 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,035 kB]
Hit:13 https://ppa.launchpadcontent.ne

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_402"
OpenJDK Runtime Environment (build 1.8.0_402-8u402-ga-2ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.402-b06, mixed mode)


In [3]:
import findspark
findspark.init()


In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession #para poder usar dataframes
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
#para el formato y tipos de datos que usaremos
from pyspark.sql.types import Row #para formato de los tipos de columnas
from pyspark.sql import SQLContext #para usar las operaciones propias de sql y dataframes

In [5]:
sc = SparkContext(master="local", appName="DataFrames")
sqlContext = SQLContext(sc)

/content/spark-3.5.1-bin-hadoop3/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
path = "/content/gdrive/My Drive/DatosSparkBD/"

In [11]:
#crear desde RDD un dataframe
deportistaOlimpicoRDD = sc.textFile(path+"deportista.csv").map(lambda linea: linea.split(","))

In [12]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [13]:
#Función para eliminar encabezados
def eliminaEncabezado(index,iterator):
  return iter(list(iterator)[1:])

In [14]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

In [15]:
deportistaOlimpicoRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [16]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda e: (
    int(e[0]),
    e[1],
    int(e[2]),
    int(e[3]),
    int(e[4]),
    float(e[5]),
    int(e[6])
    ))

In [ ]:
#['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],

In [17]:
schema = StructType([
    StructField("deportista_id",IntegerType(),False),
    StructField("nombre",StringType(),False),
    StructField("genero",IntegerType(),False),
    StructField("edad", IntegerType(), False),
    StructField("altura", IntegerType(), False),
    StructField("peso", FloatType(),False),
    StructField("equipo_id",IntegerType(),False)
])

In [19]:
sqlContext.createDataFrame(deportistaOlimpicoRDD,schema).show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [20]:
deportistaDF = sqlContext.createDataFrame(deportistaOlimpicoRDD,schema)

In [21]:
deportistaDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [ ]:
#['', 'nombre_juego', 'annio', 'temporada', 'ciudad']

In [22]:
#crear dataframe desde archivo
juegoSchema = StructType([
    StructField("juego_id",IntegerType(),False),
    StructField("nombre_juego",StringType(),False),
    StructField("annio", IntegerType(),False),
    StructField("temporada",StringType(),False),
    StructField("ciudad",StringType(),False)
])

In [23]:
juegosDF = sqlContext.read.schema(juegoSchema).option("header","true").csv(path+"juegos.csv")

In [24]:
juegosDF.show(5)

+--------+------------+-----+---------+---------+
|juego_id|nombre_juego|annio|temporada|   ciudad|
+--------+------------+-----+---------+---------+
|       1| 1896 Verano| 1896|   Verano|   Athina|
|       2| 1900 Verano| 1900|   Verano|    Paris|
|       3| 1904 Verano| 1904|   Verano|St. Louis|
|       4| 1906 Verano| 1906|   Verano|   Athina|
|       5| 1908 Verano| 1908|   Verano|   London|
+--------+------------+-----+---------+---------+
only showing top 5 rows



In [25]:
juegosDF.printSchema()

root
 |-- juego_id: integer (nullable = true)
 |-- nombre_juego: string (nullable = true)
 |-- annio: integer (nullable = true)
 |-- temporada: string (nullable = true)
 |-- ciudad: string (nullable = true)



In [26]:
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [27]:
deportistaDF = deportistaDF.withColumnRenamed("genero","sexo")

In [28]:
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [29]:
deportistaDF = deportistaDF.drop("altura")

In [30]:
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [31]:
from pyspark.sql.functions import *



In [32]:
deportistaDFSQL = deportistaDF.select("deportista_id","nombre",col("edad").alias("edadAlJugar"),"equipo_id")

In [33]:
deportistaDFSQL.show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [35]:
deportistaDFSQL.sort("edadAlJugar").show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|          133|           Franz Abb|          0|      399|
|          167|Ould Lamine Abdallah|          0|      362|
|           66|     Mohamed Abakkar|          0|     1003|
|          163|     Ismail Abdallah|          0|     1095|
|          139|George Ioannis Abbot|          0|     1043|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [36]:
deportistaDFSQL  = deportistaDFSQL.filter((deportistaDFSQL.edadAlJugar != 0))

In [ ]:
#deportistaDFSQL  = deportistaDFSQL.filter((deportistaDFSQL.edadAlJugar != 0) || (#otra condición))

In [37]:
deportistaDFSQL.sort("edadAlJugar").show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        22411|Magdalena Cecilia...|         11|      413|
|        40129|    Luigina Giavotti|         11|      507|
|        37333|Carlos Bienvenido...|         11|      982|
|        47618|Sonja Henie Toppi...|         11|      742|
|        51268|      Beatrice Hutiu|         11|      861|
+-------------+--------------------+-----------+---------+
only showing top 5 rows

